In [ ]:
# REFERENCES:
# https://medium.com/botsbrasil/como-reconhecer-entidades-na-l%C3%ADngua-portuguesa-usando-spacy-8a5ca6f42c4f
# https://medium.com/@van3ssabandeira/o-famoso-spacy-90afb683b6fe
# https://towardsdatascience.com/clinical-named-entity-recognition-using-spacy-5ae9c002e86f
# https://www.machinelearningplus.com/spacy-tutorial-nlp/
# https://tede2.pucrs.br/tede2/bitstream/tede/9970/2/DANIEL%20ALESSANDRO%20GUIMAR%C3%83ES%20DE%20LOS%20REYES_DIS.pdf
# https://towardsdatascience.com/clinical-named-entity-recognition-using-spacy-5ae9c002e86f
# https://www.neuratec.com/blog/?p=285
# https://stackoverflow.com/questions/72772448/spacy-adding-multiple-patterns-to-a-single-ner-using-entity-ruler
# https://github.com/explosion/spaCy/issues/4042
#
# https://web.stanford.edu/%7Elaurik/publications/archive/questions.pdf
#
# https://stackoverflow.com/questions/67991549/how-to-extract-questions-from-text-using-spacy
# https://stackoverflow.com/questions/72819016/extract-information-from-a-text-using-spacy
# https://stackoverflow.com/questions/66671292/can-i-identify-question-wordswho-how-when-in-a-string-using-spacy
# https://www.analyticsvidhya.com/blog/2021/11/end-to-end-question-answering-system-using-nlp-and-squad-dataset/
# https://towardsdatascience.com/building-a-question-answering-system-part-1-9388aadff507
# https://medium.com/casl-project/building-a-question-answering-system-part-3-answer-extraction-5b9a4bd31e18
# https://medium.com/saarthi-ai/building-intelligent-question-answering-systems-with-elmo-1c703af385cc
#
# https://newscatcherapi.com/blog/how-to-annotate-entities-with-spacy-phrase-macher
# 

In [ ]:
#!pip uninstall spacy 

In [ ]:
# INSTALATIONS
#!pip install -U transformers==4.13.0
#!pip install -U spacy  # Successfully installed spacy-3.4.1
#!python -m spacy download pt
#!pip install scispacy
#!pip install git+https://github.com/explosion/spacy-transformers
#!pip install spacy[transformers]
#!pip install spacy-transformers

In [1]:
#import spacy

In [ ]:
#nlp = spacy.load("pt_core_news_sm")
#nlp = spacy.load("pt")
#import pt_core_news_sm
#nlp = pt_core_news_sm.load()

In [2]:
#nlp = spacy.load('pt_core_news_sm')

In [43]:
path = "/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFsProcessing/txts/"

file = path+"01001949620195010225-DEJT-27-02-2020.pdf.txt"

arq = open(file, encoding = 'utf8')


In [310]:
import re

def get_campos(lines):
    
    
    pattern_reclamante = r"(RECLAMANTE:|Reclamante:).*(RECLAMADO:|Reclamado)"
    pattern_reclamado = r"(RECLAMADO:|Reclamado:).*(SENTENÇA|Sentença)|(RECLAMADO:|Reclamado:).*(RELATÓRIO|Relatório)"
    pattern_relatorio = r"(RELATÓRIO|Relatório).*(FUNDAMENTAÇÃO|Fundamentação)|(RELATÓRIO|Relatório).*(relatório)"
    pattern_fundamentacao = r"(FUNDAMENTAÇÃO|Fundamentação).*(DISPOSITIVO|Dispositivo)"
    pattern_dispositivo = r"(DISPOSITIVO|Dispositivo).*|(Isto posto,).*"
    pattern_vara = r"(REGIÃO).*(Vara do Trabalho)"
    pattern_processo = r"(PROCESSO:|Processo).*(CLASSE:|Classe)|(ATOrd).*(RECLAMANTE:|Reclamante)"
    
    try:
        reclamante = re.search(pattern_reclamante, lines).group(0).replace("RECLAMANTE:","").replace("RECLAMADO:","")
    except:
        try:
            pattern_reclamante = r"(Relatório ).*?(, |ajuizou)|(, ).*(, Reclamante)"
            reclamante = re.search(pattern_reclamante, lines).group(0).replace("Relatório ","").replace("ajuizou","").replace(", Reclamante","")
            reclamante = reclamante[reclamante.rfind(","):len(reclamante)].replace(",","")
        except:
            try:
                pattern_reclamante = r"(SENTENÇA|Sentença).*?(,)"
                reclamante = re.search(pattern_reclamante, lines).group(0).replace("SENTENÇA ","").replace("Sentença","").replace(",","")
            except:
                reclamante = ""
        
    try:
        reclamado = re.search(pattern_reclamado, lines).group(0).replace("RECLAMADO:","").replace("SENTENÇA","").replace("RELATÓRIO","")
    except:
        try:
            pattern_reclamado = r"(em face de ).*?(, )|(, e).*(, Reclamad)"
            reclamado = re.search(pattern_reclamado, lines).group(0).replace("em face de ","").replace(",","").replace("Reclamad","")
        except:
            reclamado = ""

    try:
        vara = re.search(pattern_vara, lines).group(0).replace("REGIÃO","").replace("Vara do Trabalho","")
    except:
        vara = ""

    try:
        processo = re.search(pattern_processo, lines).group(0).replace("PROCESSO:","").replace("CLASSE:","").replace("ATOrd","").replace("RECLAMANTE:","")
    except:
        processo = ""

    try:
        relatorio = re.search(pattern_relatorio, lines).group(0).replace("RELATÓRIO","").replace("FUNDAMENTAÇÃO","").replace("Relatório","").replace("Fundamentação","")
    except:
        try:
            pattern_relatorio = r"(ajuizou).*( relatório)"
            relatorio = re.search(pattern_relatorio, lines).group(0).replace("relatóri","")
        except:
            relatorio = ""

    try:
        fundamentacao = re.search(pattern_fundamentacao, lines).group(0).replace("DISPOSITIVO","").replace("FUNDAMENTAÇÃO","").replace("Dispositivo","").replace("Fundamentação","")
    except:
        # Fundamentacao = Relatorio:Dispositivo - Relatorio - Dispositivo
        try:
            pattern_fundamentacao = r"(RELATÓRIO).*(DISPOSITIVO)|(DECIDO|Decido).*(Isto posto,)"
            fundamentacao = re.search(pattern_fundamentacao, lines).group(0).replace(relatorio,"").replace("RELATÓRIO","").replace("DISPOSITIVO","").replace("Relatório","").replace("Dispositivo","").replace("DECIDO","").replace("Decido","")
        except:
            fundamentacao = ""
        
        # Relatorio:Dispositivo.replace(Relatorio,"")
        # Relatorio:Dispositivo.replace("DISPOSITIVO","")

    try:
        dispositivo = re.search(pattern_dispositivo, lines).group(0).replace("DISPOSITIVO","").replace("Dispositivo","")
        juiz  = dispositivo[dispositivo.rfind("."):len(dispositivo)].replace(".","")

    except:
        dispositivo = ""
        juiz = ""

        
    return (reclamante, reclamado, processo, vara, juiz, relatorio, fundamentacao, dispositivo)

In [311]:
import os
#directoryList = os.listdir(path)
directoryList = []
# Funciona:
#directoryList.append("01010991020205010341-DEJT-05-01-2022.pdf.txt")
#directoryList.append("00124718620155010481-DOERJ-07-06-2017.pdf.txt")
#directoryList.append("00116975620155010481-10-07-2016.pdf.txt")
#
#directoryList.append("01017989320175010021-DOERJ-17-05-2018.pdf.txt")
#directoryList.append("01006796620205010062-DEJT-27-10-2020.pdf.txt")
#directoryList.append("01024495220175010401-DEJT-20-08-2021.pdf.txt")
#directoryList.append("01002601020185010032-DEJT-14-04-2018.pdf.txt")
#
# Nao funciona:
#directoryList.append("00111906420145010050-08-10-2015.pdf.txt") # ATA DE AUDIÊNCIA. Vale a pena descarta-la!
#directoryList.append("01003898420185010009-DEJT-15-05-2018.pdf.txt") # Documento pequeno
#directoryList.append("01006042520185010053-DEJT-12-12-2018.pdf.txt") # Dificil de entender. Sem estrutura
#
#directoryList.append("")


sentenca   = []
reclamante = []
reclamado  = []
vara       = []
processo   = []
juiz       = []
relatorio  = []
fundamentacao = []
dispositivo   = []
classificacao = []

for i in range(0,len(directoryList)):
    print('i: ',i,"file: ",directoryList[i])
    with open(path+directoryList[i], encoding = 'utf8') as f:
        text = f.readlines()
        text = "".join(text)
        text = text.replace("\n", " ")
        sentenca.append(text)
        recl_te, recl_do, process_, vara_, juiz_, relat_, fundam_, dispos_ = get_campos(text)
        if(process_ == ""):
            process_ = os.path.splitext(os.path.splitext(directoryList[i])[0])[0]
            
    f.close()
    

    reclamante.append(recl_te)
    reclamado.append(recl_do)
    vara.append(vara_)
    processo.append(process_)
    juiz.append(juiz_)
    relatorio.append(relat_)
    fundamentacao.append(fundam_)
    dispositivo.append(dispos_)
    classificacao.append("")
    


#print(sentenca[0])
'''
print("============================")
print("Reclamante: ",reclamante[0])
print("----------------------------")
print("Reclamado: ",reclamado[0])
print("----------------------------")
print("Vara: ",vara[0])
print("----------------------------")
print("Processo: ", processo[0])
print("----------------------------")
print("Juiz: ",juiz[0])
print("----------------------------")
print("Relatorio: ",relatorio[0])
print("----------------------------")
print("Fundamentacao: ",fundamentacao[0])
print("----------------------------")
print("Dispositivo: ",dispositivo[0])
#print("Classificacao: ", classificacao[0])
print("============================")
'''

i:  0 file:  01002601020185010032-DEJT-14-04-2018.pdf.txt
PODER JUDICIÁRIO FEDERAL  JUSTIÇA DO TRABALHO  TRIBUNAL REGIONAL DO TRABALHO DA 1º REGIÃO 36º VARA DO TRABALHO DO RIO DE JANEIRO  PROCESSO nº 0100260-10.2018.5.01.0032 SENTENÇA  SINDICATO DOS EMPREGADOS EM ESTABELECIMENTOS BANCÁRIOS DO MUNICÍPIO DO RIO DE JANEIRO, qualificado, ajuizou ação civil pública em face de CAIXA ECONÔMICA FEDERAL, pleiteando pedidos, e fundando-se nas razões da petição inicial.  Êo relatório.  DECIDO.  Consoante dispõe o artigo 485, 83º, do CPC, de inquestionável aplicação na seara laboral, o juiz conhecerá de ofício, em qualquer tempo e grau de jurisdição, enquanto não proferida a sentença de mérito, a falta de interesse processual.  O conceito de interesse processual é composto pelo binômio necessidade-adequação, refletindo aquela a indispensabilidade do ingresso em juízo para a obtenção do bem da vida pretendido e se consubstanciando esta na relação de pertinência entre a situação material que se tenc

In [115]:
# 1. Para um arquivo especifico:
# Já podemos instanciar os modelos de NER, Pergunta e Resposta e Resumo (em portugues)
#   para extrair novas colunas (Fato/Assunto, Jurisprudencia/Lei)para o Analytics.
#   Como as seções das sentenças estão separadas em campos no dataset (relatorio, fundamentacao, dispositivo, etc)
#   ficara mais fácil inserir estas seções nos modelos de NLP/BERT. Podemos também usa-las no pipe do SpaCy e 
#   guardar numa coluna tempoária (ou lista) o quantitativo de tokens e depois calcular o max, min, média de tokens
#   em cada seção da sentença e se precisamos fazer padding ou não.
#
# Verificar os registros (processos/arquivos) que possuem algum campo vazio para aperfeiçoar a captura/BERT
#
# E a classificacao (procedencia)? Copiar o notebook que captura a classificacao do dispositivo
#   Excluir do dataset os registros que nao foi possivel fazer a classificacao exata.
#   É possivel fazermos a classificacao/procedencia por pedido? Como reconhecer os pedidos? Se parcialmente 
#      procedente, pedido tal procedente, pedido tal improcedente, etc.
#
# 2. Já podemos criar um Dataset com as colunas capturadas neste arquivo (antes de rodar para todos os txts da pasta)
# Verificar os registros (processos/arquivos) que possuem algum campo vazio para aperfeiçoar a captura/regex